## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [1]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [2]:
path_directorio='csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=1,day=8)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [3]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [4]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210119.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [5]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,209520,6800,216320,164820,0.761927,1773,2021-01-18
1,Aragón,35415,1200,36615,28316,0.773344,250,2021-01-18
2,Asturias,35420,1200,36620,34293,0.936455,1285,2021-01-19
3,Baleares,16740,600,17340,14658,0.845329,164,2021-01-18
4,Canarias,41310,1300,42610,43209,1.014058,339,2021-01-19
5,Cantabria,17700,500,18200,18967,1.042143,0,2021-01-19
6,Castilla y Leon,76740,2500,79240,70885,0.894561,432,2021-01-18
7,Castilla La Mancha,53100,1700,54800,43682,0.797117,189,2021-01-18
8,Cataluña,180020,5800,185820,156627,0.842896,432,2021-01-18
9,C. Valenciana,91450,2900,94350,95260,1.009645,588,2021-01-19


..hacemos una copia para hacerle las transformaciones necesarias:

In [6]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [7]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [8]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [9]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [10]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [11]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                     'Dosis entregadas Moderna','Dosis entregadas',
                     'Dosis administradas','% sobre entregadas',
                     'Total pauta completada',
                     'Fecha de la ultima vacuna registrada',
                     'source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [12]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [13]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [14]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,19/1/21,Andalucía,209520,6800,216320,164820,0.761927,1773,18/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,19/1/21,Aragón,35415,1200,36615,28316,0.773344,250,18/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,19/1/21,Asturias,35420,1200,36620,34293,0.936455,1285,19/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,19/1/21,Baleares,16740,600,17340,14658,0.845329,164,18/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,19/1/21,Canarias,41310,1300,42610,43209,1.014058,339,19/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,19/1/21,Cantabria,17700,500,18200,18967,1.042143,0,19/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,19/1/21,Castilla y Leon,76740,2500,79240,70885,0.894561,432,18/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,19/1/21,Castilla La Mancha,53100,1700,54800,43682,0.797117,189,18/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,19/1/21,Cataluña,180020,5800,185820,156627,0.842896,432,18/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,19/1/21,C. Valenciana,91450,2900,94350,95260,1.009645,588,19/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [15]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [16]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [17]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...
195,18/1/21,Navarra,17700.0,500.0,18200,11834,0.650220,0.0,NaN,17/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
196,18/1/21,País Vasco,47205.0,1400.0,48605,25526,0.525172,0.0,NaN,17/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
197,18/1/21,Ceuta,1005.0,200.0,1205,981,0.814108,29.0,NaN,17/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
198,18/1/21,Melilla,1005.0,200.0,1205,1289,1.069710,0.0,NaN,15/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [18]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [19]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas', 'Dosis administradas',
       '% sobre entregadas', 'Total pauta completada',
       'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [20]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [21]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...
215,19/1/21,Navarra,17700.0,500.0,18200,12991,0.713791,299.0,NaN,18/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
216,19/1/21,País Vasco,47205.0,1400.0,48605,28069,0.577492,390.0,NaN,18/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
217,19/1/21,Ceuta,1005.0,200.0,1205,1080,0.896266,29.0,NaN,18/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
218,19/1/21,Melilla,1005.0,200.0,1205,1297,1.076349,29.0,NaN,18/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [22]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [23]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)